In [16]:
from elasticsearch import Elasticsearch
import pandas as pd
import json

In [54]:
with open('./creds/elastic-credentials.csv') as f:
    rows = f.readlines()
CREDS = rows[1]
USERNAME = CREDS.split(',')[0].strip()
PASSWORD = CREDS.split(',')[1].strip()

# Found in the 'Manage Deployment' page
CLOUD_ID = "9b78dc4b87cb480bb9388eabc314c9f0:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQ0MDkyMzRiMmE4YTk0MjRjYTQyZjA3NTYwYzFjMjkyNCRkMmU4ZjVmNTA5MzY0ZWJhYmQ1ZWQ1NTc4MTI2NjVkZA=="

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=(USERNAME, PASSWORD)
)

# if connection is successful, displays client connection info
client.info()

7 kpfk5RF8BrIUrTi3I8Ov7S8B


ObjectApiResponse({'name': 'instance-0000000000', 'cluster_name': '409234b2a8a9424ca42f07560c1c2924', 'cluster_uuid': 'dwl2cew_RnW-rZva9Xt6Iw', 'version': {'number': '8.13.4', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'da95df118650b55a500dcc181889ac35c6d8da7c', 'build_date': '2024-05-06T22:04:45.107454559Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [39]:
# Get documents from filebeat index
result = client.search(index="filebeat-*", body={"query": {"match_all": {}}}, scroll="2m", size=100)
# Initialize scroll_id and hits
scroll_id = result['_scroll_id']
all_hits = result['hits']['hits']

# Paginate through all results
while len(result['hits']['hits']):
    print(f"scroll_id: {scroll_id}")
    result = client.scroll(
        scroll_id=scroll_id,
        scroll='2m'  # Scroll timeout (keep the context alive for 2 minutes)
    )
    all_hits.extend(result['hits']['hits'])
    scroll_id = result['_scroll_id']

# Clear the scroll context to free resources
client.clear_scroll(scroll_id=scroll_id)

# all_hits contains all documents
print(f"Total documents fetched: {len(all_hits)}")
for hit in all_hits:
    print(hit["_source"])

/var/folders/5v/gb09btt1621g_l5nvjnb26v00000gn/T/ipykernel_4687/1050685689.py:2: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  result = client.search(index="filebeat-*", body={"query": {"match_all": {}}}, scroll="2m", size=100)


scroll_id: FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFnYzSnR5b29xVE1tMmZlREh1M3dYX3cAAAAAAAJr-hZHQVRZa0RfR1F3S05CNllCSTJYbXhn
scroll_id: FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFnYzSnR5b29xVE1tMmZlREh1M3dYX3cAAAAAAAJr-hZHQVRZa0RfR1F3S05CNllCSTJYbXhn
scroll_id: FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFnYzSnR5b29xVE1tMmZlREh1M3dYX3cAAAAAAAJr-hZHQVRZa0RfR1F3S05CNllCSTJYbXhn
scroll_id: FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFnYzSnR5b29xVE1tMmZlREh1M3dYX3cAAAAAAAJr-hZHQVRZa0RfR1F3S05CNllCSTJYbXhn
scroll_id: FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFnYzSnR5b29xVE1tMmZlREh1M3dYX3cAAAAAAAJr-hZHQVRZa0RfR1F3S05CNllCSTJYbXhn
scroll_id: FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFnYzSnR5b29xVE1tMmZlREh1M3dYX3cAAAAAAAJr-hZHQVRZa0RfR1F3S05CNllCSTJYbXhn
scroll_id: FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFnYzSnR5b29xVE1tMmZlREh1M3dYX3cAAAAAAAJr-hZHQVRZa0RfR1F3S05CNllCSTJYbXhn
Total documents fetched: 602
{'agent': {'name': 'vm-2a', 'id': '6b939f3a-f2d

In [60]:
# Store result as pandas data frame
df = pd.DataFrame([doc["_source"] for doc in all_hits])
df.head()

,agent,process,log,fileset,auditd,cloud,input,@timestamp,ecs,service,host,event,user,source,message,error,network
0,"{'name': 'vm-2a', 'id': '6b939f3a-f2d6-48a0-98...","{'name': 'systemd', 'pid': 1, 'executable': '/...","{'file': {'path': '/var/log/audit/audit.log'},...",{'name': 'log'},"{'log': {'ses': '4294967295', 'record_type': '...","{'availability_zone': 'us-west1-a', 'instance'...",{'type': 'log'},2024-05-17T04:53:37.743Z,{'version': '1.12.0'},{'type': 'auditd'},"{'hostname': 'vm-2a', 'os': {'kernel': '5.15.0...","{'ingested': '2024-05-17T04:53:57.959936919Z',...","{'audit': {'id': '4294967295'}, 'id': '0'}",NaN,NaN,NaN,NaN
1,"{'name': 'vm-2a', 'id': '6b939f3a-f2d6-48a0-98...","{'name': 'systemd', 'pid': 1, 'executable': '/...","{'file': {'path': '/var/log/audit/audit.log'},...",{'name': 'log'},"{'log': {'ses': '4294967295', 'record_type': '...","{'availability_zone': 'us-west1-a', 'instance'...",{'type': 'log'},2024-05-17T04:53:37.743Z,{'version': '1.12.0'},{'type': 'auditd'},"{'hostname': 'vm-2a', 'os': {'kernel': '5.15.0...","{'ingested': '2024-05-17T04:53:57.962490079Z',...","{'audit': {'id': '4294967295'}, 'id': '0'}",NaN,NaN,NaN,NaN
2,"{'name': 'vm-2a', 'id': '6b939f3a-f2d6-48a0-98...","{'pid': 3448, 'executable': '/usr/bin/sudo'}","{'file': {'path': '/var/log/audit/audit.log'},...",{'name': 'log'},"{'log': {'ses': '14', 'op': 'PAM:session_close...","{'availability_zone': 'us-west1-a', 'instance'...",{'type': 'log'},2024-05-17T04:49:32.437Z,{'version': '1.12.0'},{'type': 'auditd'},"{'hostname': 'vm-2a', 'os': {'kernel': '5.15.0...","{'ingested': '2024-05-17T04:49:45.895448342Z',...","{'effective': {'name': 'root'}, 'audit': {'id'...",NaN,NaN,NaN,NaN
3,"{'name': 'vm-2a', 'id': '6b939f3a-f2d6-48a0-98...","{'pid': 3448, 'executable': '/usr/bin/sudo'}","{'file': {'path': '/var/log/audit/audit.log'},...",{'name': 'log'},"{'log': {'ses': '14', 'op': 'PAM:setcred', 're...","{'availability_zone': 'us-west1-a', 'instance'...",{'type': 'log'},2024-05-17T04:49:32.437Z,{'version': '1.12.0'},{'type': 'auditd'},"{'hostname': 'vm-2a', 'os': {'kernel': '5.15.0...","{'ingested': '2024-05-17T04:49:45.925580427Z',...","{'effective': {'name': 'root'}, 'audit': {'id'...",NaN,NaN,NaN,NaN
4,"{'name': 'vm-2a', 'id': '6b939f3a-f2d6-48a0-98...","{'pid': 3343, 'executable': '/usr/sbin/sshd'}","{'file': {'path': '/var/log/audit/audit.log'},...",{'name': 'log'},"{'log': {'ses': '14', 'op': 'PAM:session_close...","{'availability_zone': 'us-west1-a', 'instance'...",{'type': 'log'},2024-05-17T04:49:34.805Z,{'version': '1.12.0'},{'type': 'auditd'},"{'hostname': 'vm-2a', 'os': {'kernel': '5.15.0...","{'ingested': '2024-05-17T04:49:45.928067719Z',...","{'effective': {'name': 'professor'}, 'audit': ...","{'geo': {'region_iso_code': 'US-CA', 'continen...",NaN,NaN,NaN
